In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf

In [2]:
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': []
    }

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_test = np.argmax(y_test, axis=1) 
        print("y_test: ", y_test)
        print("y_pred: ", y_pred)
        print("history: ", history.history)
        
        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['precision'].append(precision_score(y_test, y_pred, average='macro'))
        scores['recall'].append(recall_score(y_test, y_pred, average='macro'))
        scores['f1'].append(f1_score(y_test, y_pred, average='macro'))
        scores['roc_auc'].append(roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro'))  # ROC AUC 多分类问题

    avg_scores = {key: np.mean(value) for key, value in scores.items()}
    return avg_scores

In [3]:
def load_cifar100_data(data_dir):
    with open(f"{data_dir}/meta", 'rb') as f:
        meta = pickle.load(f, encoding='latin1')
    
    with open(f"{data_dir}/train", 'rb') as f:
        train_data = pickle.load(f, encoding='latin1')
        
    with open(f"{data_dir}/test", 'rb') as f:
        test_data = pickle.load(f, encoding='latin1')
    
    return meta, train_data, test_data

In [4]:
def display_cifar100_data(meta, train_data, test_data, need_show_image=False):
    # Show Name of Classes
    print("CIFAR-100 Classes:")
    for i, label in enumerate(meta['fine_label_names']):
        print(f"{i}: {label}")
    
    # Show Training data shape and num
    print(f"\nTrain data shape: {train_data['data'].shape}")
    print(f"Number of training examples: {train_data['data'].shape[0]}")
    
    # Show Testing data shape and num
    print(f"\nTest data shape: {test_data['data'].shape}")
    print(f"Number of test examples: {test_data['data'].shape[0]}")

    # Show some images from training set if need
    if need_show_image:
        num_images = 10  # num to show
        images = train_data['data'][:num_images]
        labels = train_data['fine_labels'][:num_images]
    
        images = images.reshape(num_images, 3, 32, 32).transpose(0, 2, 3, 1)
        images = np.clip(images / 255.0, 0, 1)  

        plt.figure(figsize=(15, 5))
        for i in range(num_images):
            plt.subplot(2, 5, i + 1)
            plt.imshow(images[i])
            plt.title(meta['fine_label_names'][labels[i]])
            plt.axis('off')
        plt.tight_layout()
        plt.show()

In [5]:
data_dir = 'cifar-100-python'  # path directory
meta, train_data, test_data = load_cifar100_data(data_dir)
display_cifar100_data(meta, train_data, test_data)

CIFAR-100 Classes:
0: apple
1: aquarium_fish
2: baby
3: bear
4: beaver
5: bed
6: bee
7: beetle
8: bicycle
9: bottle
10: bowl
11: boy
12: bridge
13: bus
14: butterfly
15: camel
16: can
17: castle
18: caterpillar
19: cattle
20: chair
21: chimpanzee
22: clock
23: cloud
24: cockroach
25: couch
26: crab
27: crocodile
28: cup
29: dinosaur
30: dolphin
31: elephant
32: flatfish
33: forest
34: fox
35: girl
36: hamster
37: house
38: kangaroo
39: keyboard
40: lamp
41: lawn_mower
42: leopard
43: lion
44: lizard
45: lobster
46: man
47: maple_tree
48: motorcycle
49: mountain
50: mouse
51: mushroom
52: oak_tree
53: orange
54: orchid
55: otter
56: palm_tree
57: pear
58: pickup_truck
59: pine_tree
60: plain
61: plate
62: poppy
63: porcupine
64: possum
65: rabbit
66: raccoon
67: ray
68: road
69: rocket
70: rose
71: sea
72: seal
73: shark
74: shrew
75: skunk
76: skyscraper
77: snail
78: snake
79: spider
80: squirrel
81: streetcar
82: sunflower
83: sweet_pepper
84: table
85: tank
86: telephone
87: televis

In [6]:
def merge_train_test_data(train_data, test_data):
    X_train = train_data['data']
    y_train = train_data['fine_labels']
    
    X_test = test_data['data']
    y_test = test_data['fine_labels']
    
    X_combined = np.concatenate((X_train, X_test), axis=0)
    y_combined = np.concatenate((y_train, y_test), axis=0)
    
    return X_combined, y_combined

In [7]:
def build_simple_3d_cnn(input_shape, num_classes, learning_rate=0.0005, num_filter=16, filter_size=(3,3,3), dropout_rate=0.5):
    model = models.Sequential()
    model.add(layers.Conv3D(num_filter, filter_size, activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling3D((1, 2, 2))) 
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
input_shape = (1, 32, 32, 3)
num_classes = 100

learning_rates=[0.001, 0.0005, 0.0015, 0.01, 0.005]
num_filters=[8, 16, 32, 64, 128]
filter_sizes=[(1,1,1), (3,3,3), (5,5,5), (7,7,7), (9,9,9)]
dropout_rates=[0.3, 0.5, 0.6, 0.75, 0.9]

highest_config=[]
highest_accuracy=0
for learning_rate in learning_rates:
    for num_filter in num_filters:
        for filter_size in filter_sizes:
            for dropout_rate in dropout_rates:
                current_config = [learning_rate, num_filter, filter_size, dropout_rate]
                model = build_simple_3d_cnn(input_shape, num_classes, learning_rate, num_filter, filter_size, dropout_rate)

                X_combined, y_combined = merge_train_test_data(train_data, test_data)
                X_combined_reshaped = X_combined.reshape(-1, 1, 32, 32, 3)
                y_combined = to_categorical(y_combined, num_classes=100)
                X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

                avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined)
                print("current_config: ", current_config)
                print(avg_scores)

                if avg_scores['accuracy'] > highest_accuracy:
                    highest_config = current_config
                    highest_accuracy = avg_scores['accuracy']
                    print("highest_config: ", highest_config)
                    print("highest_accuracy: ", highest_accuracy)

print("highest_config: ", highest_config)
print("highest_accuracy: ", highest_accuracy)



Epoch 1/5


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_4" is incompatible with the layer: expected axis -1 of input shape to have value 20480, but received input with shape (128, 2048)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(128, 1, 32, 32, 3), dtype=float32)
  • training=True
  • mask=None

In [154]:
print(avg_scores)

{'accuracy': 0.4035166666666667, 'precision': 0.40174463535938365, 'recall': 0.40408168734279226, 'f1': 0.3942659829732732, 'roc_auc': 0.9404188626066154}
